In [4]:
from cluster import cluster as cl
rc = cl.start_cluster('homard', 32)

Waiting for connection file: /cita/h/home-2/dtamayo/.ipython/profile_default/security/ipcontroller-client.json


In [9]:
rc.ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31]

In [10]:
dv = rc[:]

In [11]:
%%px --local

maxorbs = 1.e7

import numpy as np
import rebound
import random
import os
import time

a1min = 0.04
a1max = 0.06
#a1 = 0.05 # semimajor axis of innermost planet in AU

mp = 5*3.e-6 # 5 earth masses
gamma = (2./3.*mp)**(1./3.) # Hill radius with factor of a taken out
imax = 1.*np.pi/180. # maximum inclination (uniform distribution)
emax = 2.e-2 # maximum eccentricity (uniform distribution)
betamin = 5.
betamax = 9.
Noutputs = 1000

In [18]:
def system(sim_id):
    random.seed(sim_id)
    
    a1 = random.uniform(a1min, a1max)
    beta1 = random.uniform(betamin, betamax)
    a2 = a1 + beta1*gamma*a1
    beta2 = random.uniform(betamin, betamax)
    a3 = a2 + beta2*gamma*a2
    
    sim = rebound.Simulation()
    sim.integrator="whfast"
    sim.G = 4*np.pi**2
    sim.exit_min_distance = a1*gamma # use smaller hill radius as exit condition
    
    
    sim.add(m=1.)
    sim.add(m=mp, a=a1, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a2, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a3, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.move_to_com()
    ps = sim.particles
    
    P0 = ps[1].P
    tmax = maxorbs * P0 # number of inner planet orbital periods to integrate
    sim.dt = P0/100.
    
    features = [maxorbs, mp, gamma]
    for p in ps[1:]:
        features = features + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f, p.x, p.y, p.z, p.vx, p.vy, p.vz]
        
    stable = [True] # assume it's stable and update if not
    
    E0 = sim.calculate_energy()
    t0 = time.time()
    
    times = np.logspace(0,np.log10(tmax),Noutputs)
    outputs = '/mnt/raid-cita/dtamayo/Stability/data/corrdata/output'+str(sim_id)+'.csv'
    start = '/mnt/raid-cita/dtamayo/Stability/data/corrdata/start'+str(sim_id)+'.bin'
    sim.save(start)
    with open(outputs, 'w') as f:
        try:
            for t in times:
                sim.integrate(t)
                f.write(str(sim.t))
                outs = []
                for p in ps[1:]:
                    outs = outs + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
                for out in outs:
                    f.write(',{0}'.format(out))
                f.write('\n')
        except rebound.Encounter as error:
            stable = [False]
            f.write(str(sim.t)) # this block added after the 1e7 dataset was generated for future use
            outs = []
            for p in ps[1:]:
                outs = outs + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
            for out in outs:
                f.write(',{0}'.format(out))
            f.write('\n')

    features = stable+[sim.t/P0]+features+[np.abs((sim.calculate_energy()-E0)/E0), time.time()-t0]
    fname = '/mnt/raid-cita/dtamayo/Stability/data/corrdata/corrdata'+str(os.getpid())+'.csv'
    with open(fname, 'a') as f:
        f.write(str(sim_id))
        for feature in features:
            f.write(',{0}'.format(feature))
        f.write('\n')
    
    end = '/mnt/raid-cita/dtamayo/Stability/data/corrdata/end'+str(sim_id)+'.bin'
    sim.save(end)
    return features

In [19]:
Nsystems=5000
res = dv.map(system, list(range(Nsystems)))

In [ ]:
res.get()

In [1]:
import pandas as pd
import numpy as np
import os

columns = ['Sim_ID', 'Stable', 'Norbits_instability', 'Norbits', 'Mplanet', 'RH/a']
for i in ['1', '2', '3']:
    columns += ['a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i, 'x'+i, 'y'+i, 'z'+i, 'vx'+i, 'vy'+i, 'vz'+i]
columns += ['Rel_Eerr', 'integ_time']

frames = []
path = '/mnt/raid-cita/dtamayo/Stability/data/corrdata/'
for f in os.listdir(path):
        if f.startswith('corrdata'):
            frames.append(pd.read_csv(path+f, names=columns, index_col=0))

df = pd.concat(frames)
df = df.sort_index()
df.tail()

,Stable,Norbits_instability,Norbits,Mplanet,RH/a,a1,P1,e1,pomega1,inc1,...,Omega3,f3,x3,y3,z3,vx3,vy3,vz3,Rel_Eerr,integ_time
Sim_ID,,,,,,,,,,,,,,,,,,,,,
4846,True,1.000000e+07,10000000.0,0.000015,0.021544,0.048905,0.010815,0.005363,2.334668,0.003582,...,-1.143127,1.228945,-0.066969,-0.011802,-0.001095,3.758829,-23.882275,-0.107816,1.732840e-09,1980.136161
4847,True,1.000000e+07,10000000.0,0.000015,0.021544,0.046508,0.010030,0.001906,-1.502837,0.016702,...,-0.080638,-0.571678,0.037646,0.052803,0.000602,-20.324929,14.251875,0.135950,1.030882e-08,2005.009526
4848,False,4.958923e+06,10000000.0,0.000015,0.021544,0.052147,0.011908,0.015159,3.946115,0.009432,...,-2.215850,-3.133661,-0.058807,0.035334,-0.000677,-12.326968,-20.516724,0.024643,1.709799e-06,1009.142488
4849,False,2.287581e+06,10000000.0,0.000015,0.021544,0.052059,0.011878,0.010468,5.393912,0.012794,...,0.838635,-2.429093,0.007546,-0.070678,-0.000331,23.281018,2.711531,-0.097192,8.622250e-07,474.362550
4850,False,2.860608e+04,10000000.0,0.000015,0.021544,0.058636,0.014199,0.009524,-0.493503,0.010781,...,-0.223103,-0.627052,0.029763,0.070964,0.001297,-21.150344,8.589961,0.063296,5.455732e-07,15.209459


In [119]:
df.to_csv(path+'1e7datasetcorr.csv', encoding='ascii')

In [ ]:
cl.shutdown_cluster('homard', rc)